In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option("display.max_columns", None)
hbo=pd.read_csv('/kaggle/input/hbo-tv-shows-documentaries-movies-as-of-2020/HBO_MAX_Content.csv')
hbo.head()                                         # 2087 rows × 75 columns

In [ ]:
hbo.columns

<strong>Nominal</strong> - title, type, rating, rest of the "genre_" and "platforms_" columns <br>
<strong>Ordinal</strong> - decade, imdb_bucket <br>
<strong>Discrete</strong> - year, imdb_score, rotten_score <br>
<strong>Continuous</strong> -  None <br>

In [ ]:
hbo.dtypes

In [ ]:
hbo.isna().sum().head(25)

<h4>NA values in the "hbo" dataframe ( first 25 cols, for the rest the values are 0.)</h4> 

<h1>Genre</h1>

<h3>'genre' is the list of all columns containing "genre_" string .</h3>

In [ ]:
genre=[]
for col in hbo.columns:
    if 'genres_' in col:
        genre.append(col)
print(genre)

In [ ]:
hbo_genre_df=hbo[genre]
hbo_genre_df

<h3>"hbo_genre" is the subset dataframe of "hbo" dataframe with all 'genre_' columns. </h3>

In [ ]:
hbo_genre=hbo_genre_df.apply(lambda row: ', '.join([col for col, b in zip(hbo_genre_df.columns, row) if b]), axis=1).reset_index()
hbo_genre.rename(columns = {0:'Genre'}, inplace = True)
hbo_genre.head()

<h3> Merging columns of "hbo_genre" into one columne 'Genre' . </h3>

In [ ]:
hbo_genre.drop('index',axis=1,inplace=True)
hbo_genre['Genre']=hbo_genre['Genre'].str.split(',')
hbo_genre.head()

In [ ]:
freq_genre=[]
for i in hbo_genre.Genre:
    if type(i)==list:
        freq_genre.append(len(i))
print(freq_genre[:10])    # first 10 elements from the list "freq_genre" 
sns.countplot(x=freq_genre)

Histogram of Size of each row list in the "hbo_genre" Dataframe. We can see that movies with one, two or three genres are more common than other movies with more than three grenres.

<h3> Converting elements of 'Genre' columne into list for using the pandas's  'explode' method. </h3>

In [ ]:
explode_genre=hbo_genre.explode('Genre')
explode_genre.reset_index(inplace = True,drop = True)
explode_genre

In [ ]:
explode_genre.Genre.isin(['',' ']).sum()

In [ ]:
nan_value = float("NaN")
plt.figure(figsize=(14,4))
explode_genre['Genre']=explode_genre['Genre'].str.replace('genres_','')
explode_genre['Genre']=explode_genre['Genre'].str.replace(' ','')
explode_genre['Genre'].replace("", nan_value, inplace=True)
explode_genre.dropna(subset = ["Genre"], inplace=True)
sns.countplot(x=explode_genre['Genre'],order=explode_genre['Genre'].value_counts().index)
plt.xticks(rotation=90)
plt.show()

<h1>Platforms</h1>

<h4>Descending Order of genre count in the dataframe . We can see that 'Drama' is the most common genre as individual or in group with others. </h4>

In [ ]:
platform=[]
for col in hbo.columns:
    if 'platforms_' in col:
        platform.append(col)
print(platform)

In [ ]:
hbo_plat=hbo[platform]
hbo_plat=hbo_plat.apply(lambda row: ', '.join([col for col, b in zip(hbo_plat.columns, row) if b]), axis=1).reset_index()
hbo_plat.rename(columns = {0:'Platform'}, inplace = True)
hbo_plat.head()

In [ ]:
hbo_plat.drop('index',axis=1,inplace=True)
hbo_plat['Platform']=hbo_plat['Platform'].str.split(',')
hbo_plat

In [ ]:
freq_plat=[]
freq2_plat=[]
for i in hbo_plat.Platform:
    if type(i)==list:
        freq_plat.append(len(i))
        if len(i)==2:
            freq2_plat.append(str(i))
print(freq_plat[:10])    # first 10 elements from the list "freq_genre" 
plt.title('Countwise share of different number of genre in the dataset')
sns.countplot(x=freq_plat)
plt.show()
plt.title('Countwise share of Two genre in the dataset')
sns.countplot(y=freq2_plat,palette = "Set2")
plt.xticks(rotation=90)
plt.show()

<h3>From above graph we can see that 'platform_hbo' and 'plaform_hbo_max'(as pair) has more share in platformwise(which is obvious). I draw the second graph because the first graph shows that most of the content is available on two platforms.</h3> 

In [ ]:
hbo_plat=hbo_plat.explode('Platform')
explode_genre.reset_index(inplace = True,drop = True)
hbo_plat

In [ ]:
hbo_plat['Platform']=hbo_plat['Platform'].str.replace('platforms_','')
hbo_plat['Platform']=hbo_plat['Platform'].str.replace(' ','')

In [ ]:
plt.figure(figsize=(14,4))
sns.countplot(x=hbo_plat['Platform'],order=hbo_plat['Platform'].value_counts().index)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(14,4))
hbo_all=hbo.sort_values('imdb_score',ascending=False).iloc[:,:]
plt.subplot(1, 2, 1)
plt.title('Whole Dataset')
sns.countplot(x=hbo_all.rating)

hbo_30=hbo.sort_values('imdb_score',ascending=False).iloc[:30,:]
plt.subplot(1, 2, 2)
plt.title('Top 30 films(according to imdb_score)')
sns.countplot(x=hbo_30.rating)
plt.subplots_adjust(left=0.1,bottom=0.1, right=0.9,top=0.9,wspace=0.2,hspace=0.4)

<h3>R/TV-MA - This program is intended to be viewed by mature, adult audiences and may be unsuitable for children under 17.<br><br>
PG/TV-PG - This program contains material that parents may find unsuitable for younger children. Parental guidance is recommended.<br><br>
G/TV-G - This program is suitable for all ages.<br><br>
PG-13/TV-14 - This program may be unsuitable for children under 14 years of age. </h3>

In [ ]:
hbo[['genres_Anime','genres_Stand_up_Talk','genres_LGBTQ','genres_Thriller','year']].groupby('year').agg(sum).plot.line(subplots=True)

Just the Share of my favourite genres

In [ ]:
hbo[['platforms_netflix','platforms_amazon_prime','year']].groupby('year').agg(sum).plot.line()

<h3>This graph shows that amazon_prime has added(or we can say invested) more movies from the very past than netflix has added</h3> The year shows the launch year of the movie ,not when it is added. 

In [ ]:
sns.set()
sns.displot(data=hbo,x='imdb_score',kind='kde',rug=True,hue='rating')
plt.title('Kernal Density estimation')
sns.displot(data=hbo,x='imdb_score',kind='ecdf')
plt.title('Empirical Cummulative distribution function')

<h5>In kde, we have to calculate the area under the graph between two points to get the proportion/probability of occurence of a number between that range(two numbers).</h5>
<h4>From two graphs we can see that more movies have imdb_score between 6 and 8 (because more area under the curve for kde and higher gradient(means higher density of data points, and if it seems flat which is from 2 to 4 and 9 to 10 ,then density of data points are low) for graph ecdf</h4>
<h5>We use ecdf to calculate the proportion upto a specific number. for example- </h5>

In [ ]:
print(hbo.imdb_score.isna().sum())    # 62 na values the imdb_score feature
len(hbo[hbo.imdb_score<=6])/(len(hbo)-62)  # here the specific value is 6, whick means proportion upto 6 ,whick is 18.7160%

In [ ]:
hbo.type.value_counts()     # 1712 value are na in 'type' column

In [ ]:
sns.countplot(data=hbo,x='type',hue='rating')

<h2>Any feedback is welcome. I am a beginner so please kindly help me and give your valuable suggestions.</h2>
<h3>Thanks Open Source Community for always helping.</h3> 